In [3]:
#using Pkg; Pkg.update()
using Pkg; Pkg.resolve()
using SplitApplyCombine
using ParallelTemperingMonteCarlo
using DelimitedFiles
using StaticArrays
using Random#, Plots

using BenchmarkTools

  No Changes to `~/Documents/ParallelTemperingMonteCarlo.jl/Project.toml`
  No Changes to `~/Documents/ParallelTemperingMonteCarlo.jl/Manifest.toml`


Below was used to benchmark generating vectors

In [2]:
# genvector1(max_displacement) = max_displacement * (rand(-0.5:0.001:0.5,3))

# genvector2(max_displacement) = SVector((rand()-0.5)*max_displacement,(rand()-0.5)*max_displacement,(rand()-0.5)*max_displacement)
# @benchmark genvector1(rand())
# @benchmark genvector2(rand())

Below is the main code to load 13 atom neon ELJ

In [4]:
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details

mc_cycles = 80000 #default 20% equilibration cycles on top

mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

Below is the comprehensive testing performed on making a vectorised displacement function

In [4]:
# function testtime(mc_states)
#     for state in mc_states
#         ran = rand(1:length(state.config.pos))
#         trial_pos =atom_displacement(state.config.pos[ran],state.max_displ[1],state.config.bc)
#     end
# end
# @benchmark testtime(mc_states)
# # rand(1:n_atoms,n_traj)
# # V = [1 2 3; 4 5 6; 7 8 9]
# # V[:,1]
# function atom_disp(mc_state,index)
#     trial_pos = atom_displacement(mc_state.config.pos[index],mc_state.max_displ[1],mc_state.config.bc)
#     return trial_pos
# end
# function atom_disp!(mc_state,index,trial_pos::SVector)
#     trial_pos = atom_displacement(mc_state.config.pos[index],mc_state.max_displ[1],mc_state.config.bc)
#     return trial_pos
# end

# indices = rand(1:n_atoms,n_traj)

# # trial_pos = atom_disp.(mc_states,indices)

# @benchmark begin   
#     indices = rand(1:n_atoms,n_traj)
#     trial_positions = atom_disp.($mc_states,indices)
# end

# @benchmark begin 
#     indices = rand(1:n_atoms,n_traj)
#     trial_positions =  [@SVector zeros(3) for i in 1:n_traj]
#     atom_disp!.($mc_states,indices,trial_positions)
# end
# function generate_displacements(mc_states,mc_params)
#     indices=rand(1:mc_params.n_atoms,mc_params.n_traj)
#     trial_positions = atom_disp.(mc_states,indices)
#     return indices,trial_positions
# end
# @benchmark generate_displacements(mc_states,mc_params)
# @benchmark dimer_energy_atom(1, mc_states[1].dist2_mat[1,:], pot)
# @benchmark dimer_energy_atom(1, mc_states[1].dist2_mat[:,1], pot)

In [5]:
@time ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false)

Total number of moves per MC cycle: 13



equilibration done


MC loop done.


[0.00033416720801801564, 0.00033570914364260417, 0.0003524447798847365, 0.00037012920798913575, 0.00040204588717741087, 0.00039740180075891016, 0.000431344071018235, 0.00044001921647319217, 0.00047813469971175665, 0.0005126303817998226, 0.0005216025888358526, 0.0005680742694791309, 0.0006145382121848079, 0.0006551149520987524, 0.000723059318566369, 0.0008106939796432607, 0.0008766172159544981, 0.0008804014391886023, 0.0009451404512690236, 0.0011646178231171063, 0.0015595668918220388, 0.00189612831308476, 0.002431935008022701, 0.00281114432440048, 0.0032229182931200625, 0.0033325897663891288, 0.0032772484577316303, 0.0031655662114001195, 0.002603787674760687, 0.0022474261120063307, 0.0020509169630187054, 0.0019455338909096453]
done
 11.466625 seconds (89.46 M allocations: 12.821 GiB, 5.16% gc time, 7.55% compilation time)


Below is the test for generating energy after displacement

In [6]:
# function get_energy_dimer(pos,i_atom,mc_state,pot)
#     dist2_new = [distance2(pos,b) for b in mc_state.config.pos]
#     dist2_new[i_atom] = 0.

#     energy = dimer_energy_atom(i_atom,dist2_new,pot)

#     return energy, dist2_new
# end

# function generate_energies(mc_states,trial_positions,indices,pot)
#     output = get_energy_dimer.(trial_positions,indices,mc_states,Ref(pot))
#     return output
# end
# # indices,trial_positions = generate_displacements(mc_states,mc_params)

# # output = generate_energies(mc_states,trial_positions,indices,pot)
# @benchmark generate_energies(mc_states,trial_positions,indices,pot)
# @benchmark begin
#    for i in 1:n_traj
#         energy_update(trial_positions[i],indices[i],mc_states[i].config,mc_states[i].dist2_mat,pot)
#   end
# end


# energyvector, dist2new = invert(get_energy_dimer.(trial_positions,indices,mc_states,Ref(pot)))
# function generate_energies2(mc_states,trial_positions,indices,pot)
#     energyvector, dist2new = invert(get_energy_dimer.(trial_positions,indices,mc_states,Ref(pot)))

#     return energyvector,dist2new
# end
# @benchmark generate_energies2(mc_states,trial_positions,indices,pot)

In [7]:
# #got some testing for RuNNer 
# function testwritingvec(file::IOStream,test::String)
#     write(file,"$test \n")
# end

# stringvec = ["testing","vector","for","strings"]
# file = open("testfile.txt","w+")
# testwritingvec.(Ref(file),stringvec)
# close(file)

MoveStrategy{1, 0, 0}()

ErrorException: type MoveStrategy has no field a